In [2]:
import sys
import pandas as pd
import itertools
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

sys.path.insert(0, '../')
import preprocess_csv as preprocess
from textrank.textrank import KeywordSummarizer

In [10]:
df = pd.read_csv('../data/topic2/trust_robot.csv')
papers = preprocess.extract_text(df)

In [14]:
def tokenizer(text):
    '''
    apply tokenizer
    '''
    cachedStopWords = stopwords.words("english")
    RegTok = RegexpTokenizer("[\w']{3,}")
    english_stops = set(stopwords.words('english'))
    tokens = RegTok.tokenize(text.lower())
    # stopwords 제외
    words = [word for word in tokens if (word not in english_stops) and len(word) > 2]
    
    stemmer = PorterStemmer()
    word_token = [stemmer.stem(i) for i in words]
    
    return word_token

In [68]:
def get_token2word(papers):
    '''
    return the list of dictionary (key: stemmized token, value: original word)
    '''
    word_token_list = []
    for text in papers:
        cachedStopWords = stopwords.words("english")
        RegTok = RegexpTokenizer("[\w']{3,}")
        english_stops = set(stopwords.words('english'))
        tokens = RegTok.tokenize(text.lower())
        # stopwords 제외
         # stopwords 제외
        words = [word for word in tokens if (word not in english_stops) and len(word) > 2]

        stemmer = PorterStemmer()
        word_token_dict = [{stemmer.stem(word):word} for word in words]
        word_token_list.append(word_token_dict)
        
    # flatten
    word_token_list = list(itertools.chain(*word_token_list))
    
    word_token_dict = dict()
    for element in word_token_list:
        word_token_dict = merge_dicts(merged, element)
    
    return word_token_dict

In [69]:
def merge_dicts(a, b):
    c = a.copy()  # a 변수를 c에 copy 한 후,
    c.update(b)   # c를 update하여 반환

    return c     

In [70]:
word_token_dict = get_token2word(papers)

In [25]:
summarizer = KeywordSummarizer(tokenize=tokenizer, min_count=2, min_cooccurrence=1)
textrank_result = summarizer.summarize(papers, topk=30)

keywords = [element[0] for element in textrank_result]
lemma_keyword = [lemmatizer.lemmatize(word_token_dict[keyword], pos='v') for keyword in keywords]
ranks = [element[1] for element in textrank_result]

textrank_df = pd.DataFrame()
textrank_df['keyword'] = lemma_keywords; textrank_df['rank'] = ranks
textrak_df.to_csv('')